In [23]:
# Dependencies.
import pandas as pd
from config import client_id, client_secret
from params import key_signatures, modes
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint

# Load variables to environment.
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret

In [24]:
# Function to obtain user's input.
def get_input():

    user_input = input('Enter the playlist URI: ')

    return user_input

In [25]:
# Function to perform API call and return results in JSON format.
def get_results(t):

    sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

    if 'https' in t:
        split = t.split('/')
        results = sp.playlist_tracks('spotify:playlist:' + split[5], offset = 100)

    else:
        results = sp.playlist_tracks(t)

    return results

In [26]:
# Function to create dict with data.
def create_df(results):
    data = {
        'title': [],
        'artist': [],
        'album': [], 
        'spotify_uri': []
    }

    for song in results['items']:
        data['title'].append(song['track']['name'])
        data['artist'].append(song['track']['artists'][0]['name'])
        data['album'].append(song['track']['album']['name'])
        data['spotify_uri'].append(song['track']['uri'])

    if results['next'] is not None:
        first_df = pd.DataFrame(data)
        second_df = pd.DataFrame(create_df(get_results(results['next'])))
        df = pd.concat([first_df, second_df])
        return df

    else:        
        df = pd.DataFrame(data)

        return df

In [27]:
results = get_results(get_input())
df = create_df(results)
df

KeyboardInterrupt: 